In [3]:
import pandas as pd
import cfscrape
from lxml import etree

In [4]:
keywords = ['DDoS attack', 'denial of service','DDoS']

In [5]:
scraper = cfscrape.create_scraper()

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'}

In [18]:
# df = pd.DataFrame()
df= pd.read_csv('20190925_acm_dl_ddos_papers.csv').drop(['Unnamed: 0'],axis=1)

In [ ]:
for keyword in keywords:
    page = 1 
    while True:
        print(page,keyword)
        
        keyword = keyword.replace(' ','+')
        url = "https://dl.acm.org/results.cfm?query=%22"+keyword+"%22&start="+str((page-1)*20)+"&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"
        print(url)
        
        scraped_html=scraper.get(url,headers=header).content
        html = etree.HTML(scraped_html)
        
        max_entries = html.xpath("//div[@class='pagerange']/text()")[0].split()[-1]
        if (page-1)*20 > int(max_entries):
            break
        
        df_temp = pd.DataFrame()
        
        #Crawler first level
        title = []
        links = []
        publication_date = []
        venue=[]

        for entry in html.xpath("//div[@class='title']/a"):
            try:
                title.append(entry.xpath("text()")[0])
            except:
                title.append("")

            try: 
                links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
            except:
                links.append("")

        for entry in html.xpath("//div[@class='source']"): 
            try:
                publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
            except:
                publication_date.append("")

            try:
                venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
            except:
                venue.append("")

        df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})
    
        #Crawler second level
        for link in df_temp['links']:
            print(link)
            # CHECK IF THE LINK ALREADY EXIST
            if len(df[df['links']==link]) >0:
                df_temp.drop(df_temp.loc[df_temp['links']==link].index, inplace=True)
                print("duplicated")
                continue
                
            
            scraped_html=scraper.get(link,headers=header).content
            html = etree.HTML(scraped_html)

            index_current = df_temp.index[df_temp['links'] == link][0]

            try:
                df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
            except:
                df_temp.at[index_current, 'link4download'] = ""

            try:
                df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
            except:
                df_temp.at[index_current, 'num_citations'] =""

            try:
                df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
            except:
                df_temp.at[index_current, 'num_downloads_cu'] =""

            try:
                df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
            except:
                try: 
                    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
                except:
                    try:
                        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
                    except:
                        df_temp.at[index_current, 'abstract'] = ""
        
        df = df.append(df_temp,ignore_index=True)
        display(df.tail(1))
        
        page = page + 1

1 DDoS attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=0&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=2662382&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3103164&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2491225&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3155161&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3274717&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2079334&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3342037&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3139946&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2813633&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3287940&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3022260&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=1146894&preflayout=flat
duplicated
https://dl.acm.org/

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
100,Network-based attacks like the distributed den...,https://dl.acm.org/ft_gateway.cfm?id=3242279&f...,https://dl.acm.org/citation.cfm?id=3242279&pre...,0,20,December 2017,Simulating DDoS attacks on the US fiber-optics...,WSC '17: Proceedings of the 2017 Winter Simula...


2 DDoS+attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=20&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=997156&preflayout=flat
https://dl.acm.org/citation.cfm?id=1189745&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3278534&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2382255&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2980289&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2381758&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3328789&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2980379&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2897866&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=1241836&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=3022336&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2512523&preflayout=flat
duplicated
https://dl.acm.org/citation.cf

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
101,Distributed denial-of-service (DDoS) is a rapi...,https://dl.acm.org/ft_gateway.cfm?id=997156&ft...,https://dl.acm.org/citation.cfm?id=997156&pref...,249,"14,538",April 2004,A taxonomy of DDoS attack and DDoS defense mec...,ACM SIGCOMM Computer Communication Review: Vol...


...

10 DDoS+attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=180&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=3071065&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2714637&preflayout=flat
duplicated
https://dl.acm.org/citation.cfm?id=2668327&preflayout=flat
https://dl.acm.org/citation.cfm?id=2671501&preflayout=flat
https://dl.acm.org/citation.cfm?id=3078491&preflayout=flat
https://dl.acm.org/citation.cfm?id=3123461&preflayout=flat
https://dl.acm.org/citation.cfm?id=3230544&preflayout=flat
https://dl.acm.org/citation.cfm?id=1879093&preflayout=flat
https://dl.acm.org/citation.cfm?id=1592689&preflayout=flat
https://dl.acm.org/citation.cfm?id=1621893&preflayout=flat
https://dl.acm.org/citation.cfm?id=1050753&preflayout=flat
https://dl.acm.org/citation.cfm?id=2875006&preflayout=flat
https://dl.acm.org/citation.cfm?id=1544053&preflayout=flat
https://dl.acm.org/citation.cfm?id=3132866&preflayout=flat
https://

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
203,The spread of a source host is the number of d...,https://dl.acm.org/ft_gateway.cfm?id=2109151&f...,https://dl.acm.org/citation.cfm?id=2109151&pre...,5,317,October 2011,Fit a compact spread estimator in small high-s...,IEEE/ACM Transactions on Networking (TON): Vol...


11 DDoS+attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=200&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=1862272&preflayout=flat
https://dl.acm.org/citation.cfm?id=2905362&preflayout=flat
https://dl.acm.org/citation.cfm?id=1851287&preflayout=flat
https://dl.acm.org/citation.cfm?id=1029627&preflayout=flat
https://dl.acm.org/citation.cfm?id=2995275&preflayout=flat
https://dl.acm.org/citation.cfm?id=3319615&preflayout=flat
https://dl.acm.org/citation.cfm?id=986658&preflayout=flat
https://dl.acm.org/citation.cfm?id=1921247&preflayout=flat
https://dl.acm.org/citation.cfm?id=3297272&preflayout=flat
https://dl.acm.org/citation.cfm?id=3011112&preflayout=flat
https://dl.acm.org/citation.cfm?id=3339998&preflayout=flat
https://dl.acm.org/citation.cfm?id=2020452&preflayout=flat
https://dl.acm.org/citation.cfm?id=2979848&preflayout=flat
https://dl.acm.org/citation.cfm?id=1815667&preflayout=flat
https://dl.acm.org/citation.cfm

,abstract,link4download,links,num_citations,num_downloads_cu,publication_date,title,venue
223,We present a basic tool for zero day attack si...,https://dl.acm.org/ft_gateway.cfm?id=2537884&f...,https://dl.acm.org/citation.cfm?id=2537884&pre...,1,140,October 2013,Automated signature extraction for high volume...,ANCS '13: Proceedings of the ninth ACM/IEEE sy...


12 DDoS+attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=220&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=3323084&preflayout=flat
https://dl.acm.org/citation.cfm?id=3134637&preflayout=flat
https://dl.acm.org/citation.cfm?id=1298292&preflayout=flat
https://dl.acm.org/citation.cfm?id=3052587&preflayout=flat
https://dl.acm.org/citation.cfm?id=3277574&preflayout=flat
https://dl.acm.org/citation.cfm?id=3230857&preflayout=flat
https://dl.acm.org/citation.cfm?id=2935674&preflayout=flat
https://dl.acm.org/citation.cfm?id=2980431&preflayout=flat
https://dl.acm.org/citation.cfm?id=3125687&preflayout=flat


In [ ]:
df.to_csv('20190925_acm_dl_ddos_papers.csv')

In [21]:
df['id'] = df['links'].apply(lambda x: x.replace("https://dl.acm.org/citation.cfm?id=","").replace("&preflayout=flat",""))

In [25]:
df['dm']=""
df['attack_type']= ""
df['defence_deployment']=""
df['not_attack_type']=""
df['attack_impact']=""
df['attack_infra']=""
df['overview']= ""

In [34]:
df[['id','abstract','overview','dm','attack_type','defence_deployment','attack_infra','attack_impact','not_attack_type']].to_csv('abstracts4classifying.csv',sep='|')

## SENDING EMAIL AFTER THE SCRIPT ABOVE

In [ ]:
def send_email():
import smtplib
    gmail_user = 'jjsantanna.script@gmail.com'
    gmail_password = 'hosRep-tibjiw-xatca2'
    sent_from = gmail_user
    to = ['j.j.santanna@utwente.nl']
    subject = 'Script Ended'
    body = "Hey bitch, take a look on the output of your script!"
    email_text = "From: %s\nTo: %s\nSubject: %s\n\n%s\n" % (sent_from, ", ".join(to), subject, body)
try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, email_text)
        server.close()
print ('Email sent!')
except:
print ('Something went wrong...')

## DEBUGGING ERROS 1ST LEVEL CRAWLER

In [38]:
url="https://dl.acm.org/results.cfm?query=%22denial+of+service%22&start=300&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"

In [39]:
scraped_html=scraper.get(url,headers=header).content
print(scraped_html)

b'\n<!doctype html>\n<head><script type="text/javascript">/* <![CDATA[ */_cf_loadingtexthtml="<img alt=\' \' src=\'/cf_scripts/scripts/ajax/resources/cf/images/loading.gif\'/>";\n_cf_contextpath="";\n_cf_ajaxscriptsrc="/cf_scripts/scripts/ajax";\n_cf_jsonprefix=\'//\';\n_cf_websocket_port=0;\n_cf_flash_policy_port=0;\n_cf_clientid=\'8BBDE6F6C1745117D2959CA97BD1497E\';/* ]]> */</script><script type="text/javascript" src="/cf_scripts/scripts/ajax/messages/cfmessage.js"></script>\n<script type="text/javascript" src="/cf_scripts/scripts/ajax/package/cfajax.js"></script>\n<script type="text/javascript" src="/cf_scripts/scripts/cfform.js"></script>\n<script type="text/javascript" src="/cf_scripts/scripts/masks.js"></script>\n<script type="text/javascript" src="/cf_scripts/scripts/cfformhistory.js"></script>\n<script type="text/javascript" src="/cf_scripts/scripts/ajax/ckeditor/ckeditor.js"></script>\n<script type="text/javascript" src="/cf_scripts/scripts/ajax/package/cfrichtexteditor.js"></

20 20 1 1


In [95]:


len(publication_date)

61

In [98]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
title = []
links = []
publication_date = []
venue=[]

for entry in html.xpath("//div[@class='title']/a"):
    try:
        title.append(entry.xpath("text()")[0])
    except:
        title.append("")
    
    try: 
        links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
    except:
        links.append("")

for entry in html.xpath("//div[@class='source']"): 
    try:
        publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
    except:
        publication_date.append("")
    
    try:
        venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
    except:
        venue.append("")

df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})

20 20 20 20


## DEBUGING ERROS 2nd LEVEL CRAWLER

In [ ]:
link = "https://dl.acm.org/citation.cfm?id=372148&preflayout=flat"

In [ ]:
scraped_html=scraper.get(link,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
index_current = 0

try:
    df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
except:
    df_temp.at[index_current, 'link4download'] = ""

try:
    df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
except:
    df_temp.at[index_current, 'num_citations'] =""

try:
    df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
except:
    df_temp.at[index_current, 'num_downloads_cu'] =""

try:
    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
except:
    try: 
        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
    except:
        try:
            df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
        except:
            df_temp.at[index_current, 'abstract'] = ""

df_temp